### Module for Scraping Reddit
Chase Davis

In [ ]:
# Import Libraries
import requests
import json
import time
from datetime import datetime
import csv

# Relevant Subreddits for collection
# r/Ethereum
# r/EthTrader
# r/EthInvestor
# r/EthAnalysis

In [ ]:
# Pick Reddit Account
user_account = '(by /u/..........)' # replace ............ with account name
hdr = {'User-Agent': 'windows:r/ethereum.single.result:v1.0' +
       user_account}
url = 'https://www.reddit.com/r/ethereum/.json'
req = requests.get(url, headers=hdr)
json_data = json.loads(req.text)


In [ ]:
# Get data in line with request limit
data_all = json_data['data']['children']
num_of_posts = 0
time_start = time.time()
while len(data_all) <= 750:
    time.sleep(2)
    last = data_all[-1]['data']['name']
    url = 'https://www.reddit.com/r/ethereum/.json?after=' + str(last)
    req = requests.get(url, headers=hdr)
    data = json.loads(req.text)
    data_all += data['data']['children']
    if num_of_posts == len(data_all):
        print num_of_posts, len(data_all)
        break
    else:
        num_of_posts = len(data_all)
print "Time: ", time.time() - time_start

In [ ]:
# print json.dumps(data_all, indent=4, sort_keys=True)
print "Number of Data Posts: ", len(data_all)

In [ ]:
# Helper functions 

def find_earliest(data):
    times = [item["created_utc"] for item in data]
    earliest = min(times)
    print "\nTime of earliest post: ", datetime.fromtimestamp(earliest).strftime('%c')

def type_fix(value):
    if type(value) == type(1) or type(value) == type(1.):
        return float(value)
    else:
        return unicode(value).encode('utf-8')

In [ ]:
# Clean data and store
wanted_keys = ["domain", "title", "media_embed", "likes", "view_count", "name", "author", "score", "selftext", "created", "created_utc", "permalink", "num_comments"]
print "JSON: ", type(data_all[0]), len(data_all[0].keys()), data_all[0].keys()
item = data_all[0]["data"]
print "Number of Features before clean: ", len(item.keys())
# print json.dumps(data_all[0], indent = 2)

data = [d["data"] for d in data_all]
new_data = []
for i, item in enumerate(data):
    new_item = {k:type_fix(v) for k,v in item.iteritems() if k in wanted_keys}
    new_data.append(new_item)
    
print "Number of Features after clean: ", len(new_data[0].keys()),'\n\nCleaned Features: ',new_data[0].keys()
find_earliest(new_data)

# Write data to csv (appending)
with open("reddit-ethereum.csv", "a") as f:
    w = csv.DictWriter(f, new_data[0].keys())
#     w.writeheader()
    w.writerows(new_data)